## To do: Do it for each class

In [ ]:
import os
import shutil
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [ ]:
from gan.train import train

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "gpu")

In [ ]:
from gan.losses import discriminator_loss, generator_loss
from gan.losses import ls_discriminator_loss, ls_generator_loss
from gan.models import Discriminator, Generator
from gan.utils import sample_noise, show_images, deprocess_img, preprocess_img

In [ ]:
batch_size = 128
scale_size = 64  # We resize the images to 64x64 for training
#creates for entire dataset
petfinder_root = 'petfinder-img/train'
#create path for each class
GAN_Class_df = pd.read_csv('Train_GAN_classbased.csv')

labels = GAN_Class_df.sort_values('class')
class_names = list(labels.Class.unique())

for i in class_names:
    os.makedirs(os.path.join('train_class_', i))

for c in class_names:
    for i in list(labels[labels['Class']==c]['Id']):
        get_image = os.path.join('petfinder-img/train/', i)
        copy_class_img = shutil.copy(get_image, 'train_class_/'+c)

In [ ]:
pf_train = ImageFolder(root=petfinder_root, transform=transforms.Compose([
  transforms.Resize(scale_size),
  transforms.ToTensor(),
]))

pf_loader_train = DataLoader(pf_train, batch_size=batch_size, drop_last=True)

In [ ]:
imgs = pf_loader_train.__iter__().next()[0].numpy().squeeze()
show_images(imgs, color=True)

In [ ]:
NOISE_DIM = 100
NUM_EPOCHS = 5
learning_rate = 0.0002

In [ ]:
D = Discriminator().to(device)
G = Generator(noise_dim=NOISE_DIM).to(device)

In [ ]:
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas = (0.5, 0.999))
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas = (0.5, 0.999))

In [ ]:
# original gan
train(D, G, D_optimizer, G_optimizer, discriminator_loss,
          generator_loss, num_epochs=NUM_EPOCHS, show_every=150,
          train_loader=pf_loader_train, device=device)


In [ ]:
NUM_EPOCHS = 15
D = Discriminator().to(device)
G = Generator(noise_dim=NOISE_DIM).to(device)

In [ ]:
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas = (0.5, 0.999))
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas = (0.5, 0.999))

In [ ]:
# ls-gan
train(D, G, D_optimizer, G_optimizer, ls_discriminator_loss,
          ls_generator_loss, num_epochs=NUM_EPOCHS, show_every=200,
          train_loader=pf_loader_train, device=device)